# Q1. dlt Version

In this homework, we will load the data from our FAQ to Qdrant

Let's install dlt with Qdrant support and Qdrant client:

```bash
pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"
```

What's the version of dlt that you installed?

In [1]:
!uv add -q "dlt[qdrant]"

In [2]:
!dlt --version

dlt 1.20.0


# Q2. dlt pipeline

Now let's create a pipeline. 

We need to define a destination for that. Let's use the `qdrant` one:

```python
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)
```

In this case, we tell dlt (and Qdrant) to create a folder with
our data, and the name for it will be `db.qdrant`

Let's run it:

```python
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)
```

How many rows were inserted into the `zoomcamp_data` collection?

Look for `"Normalized data for the following tables:"` in the trace output.

In [ ]:
import requests
import dlt
from dlt.destinations import qdrant


@dlt.resource
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc


qdrant_destination = qdrant(
    qd_path="db.qdrant",
)


pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Run started at 2026-01-11 20:59:39.948935+00:00 and COMPLETED in 2.46 seconds with 4 steps.
Step extract COMPLETED in 0.64 seconds.

Load package 1768165180.922401 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.05 seconds.
Normalized data for the following tables:
- _dlt_pipeline_state: 1 row(s)
- zoomcamp_data: 948 row(s)

Load package 1768165180.922401 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 0.80 seconds.
Pipeline zoomcamp_pipeline load step completed in 0.79 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /Users/hyunwoooh/llm-zoomcamp-homework/workshops/dlt/db.qdrant location to store data
Load package 1768165180.922401 is LOADED and contains no failed jobs

Step run COMPLETED in 2.46 seconds.
Pipeline zoomcamp_pipeline load step completed in 0.79 seconds
1 load package(s) were loa

# Q3. Embeddings

When inserting the data, an embedding model was used. Which one?

You can find this out by inspecting the `meta.json` file created
in the target folder. During the data insertion process, a folder named db.qdrant will be created, and the meta.json file will be located inside this folder.


In [11]:
import json
from pprint import pprint

with open("db.qdrant/meta.json", "r") as f_in:
    pprint(json.load(f_in))

{'aliases': {},
 'collections': {'zoomcamp_tagged_data': {'hnsw_config': None,
                                          'metadata': None,
                                          'on_disk_payload': None,
                                          'optimizers_config': None,
                                          'quantization_config': None,
                                          'replication_factor': None,
                                          'shard_number': None,
                                          'sharding_method': None,
                                          'sparse_vectors': None,
                                          'strict_mode_config': None,
                                          'vectors': {'fast-bge-small-en': {'datatype': None,
                                                                            'distance': 'Cosine',
                                                                            'hnsw_config': None,
                            